<a href="https://colab.research.google.com/github/aharrisonau/Task-4.1D/blob/main/Task_4_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SIT796 Reinforcement Learning
##Distinction Task 4.1: Dynamic programming and its application to RL

This task follows up 1.2C, which was the definition of a truck trailer reversing problem.

In this task, the aim is to set up a policy value mapping for truck reversal.  To achieve this, the state space has been mapped to 4 variables;
- pivot X [0, 1, 2,... 30] metres
- pivot Y [-10, -9, -8, .... 30] metres
- prime mover angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians
- trailer angle [0, 2pi/10, 4pi/10, ... 18pi/10] radians

This gives 31 x 41 x 10 x 10 = 127,
100 individual co-ordinates, which is a balance between space state size and resolution

Reward of 1 is achieved with;
- pivot X,Y = (10,0)
- trailer angle = 0
- pm angle = [16pi/10, 18pi/10, 0, 2pi/10, 4pi/10]  NOTE: other PM angles represent a jack-knifed trailer and should not occur


In [2]:
# import the environment
# helper code from https://stackoverflow.com/questions/55835318/how-to-create-and-use-a-custom-openai-gym-environment-on-google-colab
%%capture
!rm -r ReverseTruck_env
!git clone https://github.com/aharrisonau/gym-ReverseTruck.git
!pip install -e gym-ReverseTruck


After running the block above, restart the runtime, otherwise the module wont be found

In [1]:
import gym_ReverseTruck

In [2]:
# preliminaries
import pandas as pd
import numpy as np
import math as math
import gym
from gym import error, spaces, utils
from gym.utils import seeding
pi = math.pi

Then we can create the environment

Note that the environment requires a truck starting position, a truck setup file and a dictionary of obstacles passed to it

In [3]:

# the initial truck model has a prime mover length of 5m and a trailer of 10m
# width of both is set as 2.5m, but not used yet
TruckDefinition = np.array( [5.0, 2.5, 10.0, 2.5])

# The truck starts with the back of the truck 20m out and 20m forward of the 
# end point (0,0) and perpendicular to the finish position
# Note that the position is consistent with the truck definition
StartPosition = np.array( [20.0,30.0,pi/2,pi/2])

# No obstacles are used yet
Obstacles = {}


In [4]:
env = gym.make('ReverseTruck-v0',TruckDefinition=TruckDefinition, StartPosition=StartPosition, Obstacles=Obstacles)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [55]:
# the calculations will be done in numpy for speed
# first define the State Space co-ordinated for the array
SSpivotX = np.arange(0,31)
SSpivotY = np.arange(-10,31)
SSpmAngle = np.linspace(0,2*pi,21)
SStrlAngle = np.linspace(0,2*pi,21)



In [6]:
# define a function that takes a start state and gets the next state

def nextStateFN(state, action, env): # function is a modified version of the environment step function

    o=env.setState(state)
    o, r, d, _ = env.step(action)
    return o, r


In [58]:
sI=[18,11,0,-4]
steerI=0
state=[SSpivotX[sI[0]],SSpivotY[sI[1]],SSpmAngle[sI[2]],SStrlAngle[sI[3]]]
steer = steering[steerI]

In [59]:
print(state)
nextStateFN(state,(move,steer),env)

[18, 1, 0.0, 5.340707511102648]


(array([17.        ,  1.        ,  0.20135792,  5.33261725]), 0.0)

In [38]:
print("Current State:",state)
nextState, reward = nextStateFN(state,(move,steer),env)
print("Next State:",nextState, reward)
print("Dictionary next State:",nextStateDict[SSpivotX[sI[0]]][SSpivotY[sI[1]]][SSpmAngle[sI[2]]][SStrlAngle[sI[3]]][steer]["nextSS"])
print("Transition reward State:",nextStateDict[SSpivotX[sI[0]]][SSpivotY[sI[1]]][SSpmAngle[sI[2]]][SStrlAngle[sI[3]]][steer]["reward"])
print("Value Current State:",nextStateDict[SSpivotX[sI[0]]][SSpivotY[sI[1]]][SSpmAngle[sI[2]]][SStrlAngle[sI[3]]]["value"])
next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
print("Next state value:",nextStateDict[next_pivX][next_pivY][next_pmAngle][next_trlAngle]["value"])

Current State: [12, 0, 0.0, 0.0]
Next State: [11.  0.  0.  0.] 0.0
Dictionary next State: [11, 0, 0.0, 0.0]
Transition reward State: 0.0
Value Current State: 0.02020408163265306
Next state value: 0.14285714285714285


In [28]:
SSpivotX[np.searchsorted(binsPivotX,9.6)]


10

In [ ]:
SSpivotY[np.searchsorted(binsPivotY,next_pivY)]

Now we step through every state in the state space and use the nextState function to get the following state for a given action.

The actions that we will use are;
move = constant -1 (which is moving 2m back)
steer = [-1, -0.2, 0, 0.2, 1] (corresponding from full lock right to full lock left)

We will check if the next state is a terminal state (out of bounds) or has a reward as well and record those results

These can then be used for value iteration for the episodic event chain

In [60]:
# set up some bins to pull state calcuations into the nearest state space points
binsPivotX = SSpivotX+0.5
binsPivotY = SSpivotY+0.5
binsPmAngle = SSpmAngle+pi/10
binsTrlAngle = SStrlAngle+pi/10

In [61]:
# set the allowable actions
move = -1.0 #only going backwards at max rate
steering = np.array([-1.0, -0.5, -0.2, 0, 0.2, 0.5, 1.0])

## State transition and rewards dictionary
With the discretized state space, we can use the Next State function to populate a dictionary that records the state transition and rewards for every state and action combination in the state and action spaces of the MDP

In [62]:
# use a nested dictionary structure to store the results
# levels will be
# 1 - PivotX bins
# 2 - PivotY bins
# 3 - PmAngle bins
# 4 - TrlAngle bins
# 5 - action index

nextStateDict = {}

for pivX in SSpivotX:
  print(pivX,",",end=" ") # heartbeat to follow progress
  nextStateDict[pivX] = {}
  for pivY in SSpivotY:
    nextStateDict[pivX][pivY] = {}
    for pmAngle in SSpmAngle:
      nextStateDict[pivX][pivY][pmAngle] = {}
      for trlAngle in SStrlAngle:
        nextStateDict[pivX][pivY][pmAngle][trlAngle] = {}
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=0.0
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=0
        # note that the value of the state and the best policy actions
        # fields are created for later use 
        for steer in steering:
          # this is now one of the actions applied at a particular state
          nextStateDict[pivX][pivY][pmAngle][trlAngle][steer] = {}
          state = [pivX,pivY,pmAngle,trlAngle]
          nextState, reward = nextStateFN(state,(move,steer),env)
          next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
          
          # check if the state has gone out of bounds.  If so, zero it out
          if  not SSpivotX[0] <= next_pivX <= SSpivotX[-1]:
            nextState = 0
          elif not SSpivotY[0] <= next_pivY <= SSpivotY[-1]:
            nextState = 0
          else:
            # we need to round the result to the nearest state space
            next_pivX = SSpivotX[np.searchsorted(binsPivotX,next_pivX)]
            next_pivY = SSpivotY[np.searchsorted(binsPivotY,next_pivY)]
            next_pmAngle = SSpmAngle[np.searchsorted(binsPmAngle,next_pmAngle)]
            next_trlAngle = SStrlAngle[np.searchsorted(binsTrlAngle,next_trlAngle)]
            nextState = [next_pivX,next_pivY,next_pmAngle,next_trlAngle]

          if nextState == 0:
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = 0
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = 0
          else:
            # assign that reward for that action at that state
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"] = reward
            # record what the next state for that action on this state is                                             
            nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"] = nextState
            

0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 

## Value Iteration ##
Now we can use a state value iteration process, to pass through all the states and update the best achievable value from the options.

A discount factor is applied so that quicker achievement of the goal is valued.

Value Iteration requires a policy for application and in this case, the policy is an even, random selection from the discretized set of possible actions.

This pass is repeated until sufficient convergence is achieved.

In [63]:
# loop through the state space and update the rewards
stillConverging = True
count = 0
gamma = 0.99  # discount factor
changeLimit = 0.000000001
actionProb = 1.00/len(steering)

while stillConverging:
  
  change = 0.0

  for pivX in SSpivotX:
    print(pivX,",",end=" ") # heartbeat to follow progress
    for pivY in SSpivotY:
      for pmAngle in SSpmAngle:
        for trlAngle in SStrlAngle:
          # reset variables to track the best value for this state
          # the best action is also updated so that once converged
          # the optimum policy can be output
          bestValue = 0.0
          bestAction = 0
          # loop through the actions
          for steer in steering:
            nextState = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"]
            reward = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"]

            if nextState != 0: # if it is zero, that action was out-of-bounds
              
              next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
              nextStateValue = nextStateDict[next_pivX][next_pivY][next_pmAngle][next_trlAngle]["value"]
              totalValue = actionProb * (reward + gamma * nextStateValue)

              if totalValue > bestValue: # if this action was better
                bestValue = totalValue   # update the best value
                bestAction = steer        # update the best action
          
          oldValue = nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]
          increase = max(bestValue - oldValue, 0.0)
          change = max(increase,change) # check if this value increase is the biggest
          if increase > 0:  
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=bestValue
            #if oldValue>0.1:
              #print([pivX,pivY,pmAngle,trlAngle], bestValue, oldValue)
            nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=bestAction

  print(count,change)
  count += 1 # update the loop count
  stillConverging = change > changeLimit # stop the loop once the largest change is small

0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 0 0.14285714285714285
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 1 0.02020408163265306
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 2 0.0028574344023323613
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 3 0.0004041228654727196
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 4 5.715451954542749e-05
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 5 8.0832820499

Once the value and best action map is complete, it can be examined.  As it is four dimensional, it is best viewed as a map of the x-y co-ords for a fixed prime mover and trailer angle set.  In this case, the map is for angles of 4*pi/10

In [64]:
map = pd.DataFrame(index=SSpivotY)

for pivX in SSpivotX:
  col = []
  for pivY in SSpivotY:
    pmAngle = SSpmAngle[0]
    trlAngle = SStrlAngle[0]
    col.append(nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"])

  map[pivX] = col

pd.options.display.float_format = '{:.2e}'.format
map


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
-10,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-9,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-8,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-7,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-6,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-5,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-4,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-1,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00


## Policy Iteration ##
Now we can modify the process to use a Policy interation process. To do this, we first loop through all the states to calculate a value for an intial policy.

A discount factor is applied so that quicker achievement of the goal is valued.

Then we loop through all the states and update the policy to select the greedy option.

Then we continue the evaluation pass and the improvement pass until the policy stabilizes.

In [81]:
# first reset the values and actions to run policy interation

for pivX in SSpivotX:
  print(pivX,",",end=" ") # heartbeat to follow progress
  for pivY in SSpivotY:
    for pmAngle in SSpmAngle:
      for trlAngle in SStrlAngle:
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=0.0
        nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=0
        # note that this Best Action value will be the starting policy

0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 

In [82]:
# loop through the state space and update the values
# note that the initial assumed policy is steering = 0 (full lock)

policyChanged = True
count = 0
gamma = 0.99  # discount factor
changeLimit = 0.001

while policyChanged:
    valuesConverging = True
    #POLICY EVALUATION LOOP
    while valuesConverging:      
      change = 0.0      
      for pivX in SSpivotX:
        print(pivX,",",end=" ") # heartbeat to follow progress
        for pivY in SSpivotY:
          for pmAngle in SSpmAngle:
            for trlAngle in SStrlAngle:
              # get the value for the current policy
              steer=nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]
              nextState = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"]
              reward = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"]
              if nextState != 0: # if it is zero, that action was out-of-bounds so do nothing
                next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
                nextStateValue = nextStateDict[next_pivX][next_pivY][next_pmAngle][next_trlAngle]["value"]
                newValue = (reward + gamma * nextStateValue)
                # note that the policy action is deterministic and so p(s'|a) = 1
                oldValue = nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]
                nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"]=newValue
                change = max(change,abs(newValue-oldValue)) # check if this value change is the biggest
      print("Count",count,change)
      count += 1 # update the loop count
      valuesConverging = change > changeLimit # stop the loop once the largest change is small

    #POLICY IMPROVEMENT
    policyChanged=False
    for pivX in SSpivotX:
      print(pivX,",",end=" ") # heartbeat to follow progress
      for pivY in SSpivotY:
        for pmAngle in SSpmAngle:
          for trlAngle in SStrlAngle:
            # reset variables to track the best value for this state
            # the best action is also updated so that once converged
            # the optimum policy can be output
            bestValue = 0.0
            bestAction = 0
            # loop through the actions
            for steer in steering:
              nextState = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["nextSS"]
              reward = nextStateDict[pivX][pivY][pmAngle][trlAngle][steer]["reward"]

              if nextState != 0: # if it is zero, that action was out-of-bounds, so do nothing
                
                next_pivX,next_pivY,next_pmAngle,next_trlAngle = nextState
                nextStateValue = nextStateDict[next_pivX][next_pivY][next_pmAngle][next_trlAngle]["value"]
                totalValue = (reward + gamma * nextStateValue)

                if totalValue > bestValue: # if this action was better
                  bestValue = totalValue   # update the best value
                  bestAction = steer        # update the best action
            
            oldAction = nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]
            if oldAction != bestAction:  
              policyChanged = True
              nextStateDict[pivX][pivY][pmAngle][trlAngle]["bestAction"]=bestAction


0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 0 1.0
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 1 0.99
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 2 0.9801
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 3 0.9702989999999999
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 4 0.96059601
0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , Count 5 0.9509900498999999
0 , 1 , 2 , 3 , 4

Once the value and best action map is complete, it can be examined.  As it is four dimensional, it is best viewed as a map of the x-y co-ords for a fixed prime mover and trailer angle set.  In this case, the map is for angles of 4*pi/10

In [83]:
map = pd.DataFrame(index=SSpivotY)

for pivX in SSpivotX:
  col = []
  for pivY in SSpivotY:
    pmAngle = SSpmAngle[0]
    trlAngle = SStrlAngle[0]
    col.append(nextStateDict[pivX][pivY][pmAngle][trlAngle]["value"])

  map[pivX] = col

pd.options.display.float_format = '{:.2e}'.format
map


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
-10,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-9,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-8,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-7,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-6,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-5,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-4,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
-1,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
